In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, Statistics, MLJ, Plots, Clustering, Distances

In [2]:
"""
J is the number of points/record in the dataset
I is the number of clusters
D is the dimensionality of the points (number of features)
"""

function generate_points(I,J,D, std, seed = 1234)
    Random.seed!(seed);
    X, yy = make_blobs(J, D; centers=I, cluster_std=std)
    points = Matrix(DataFrame(X));
    min = minimum(points, dims=1);
    max = maximum(points, dims=1);
    points = (points .- min) ./ (max .- min);
    return points
end

function manhattan_optimal_kmenas(points, I, J, D)
    
    model = JuMP.Model(Gurobi.Optimizer);

    max_d = maximum(manhattan_distance(points))

    set_optimizer_attribute(model, "TimeLimit", 300)

    @variable(model, gamma[1:J]);
    @variable(model, z[1:J, 1:I],Bin);
    @variable(model, mu[1:J, 1:I] >=0);
    @variable(model, r[1:J, 1:I] >=0);
    @variable(model, max_d >= x[1:I, 1:D] >=0);
    @variable(model, y[1:J, 1:D, 1:I] >=0);

    @constraint(model, [j = 1:J], sum(z[j,:]) == 1);

    @constraint(model, [i=1:I, j=1:J, d=1:D], sum(y[j,d,i] for d=1:D) <= r[j,i]);
    @constraint(model, [i=1:I, j=1:J, d=1:D], y[j,d,i] >= x[i,d] - points[j,d]);
    @constraint(model, [i=1:I, j=1:J, d=1:D], y[j,d,i] >= -(x[i,d] - points[j,d]));

    @constraint(model, [i=1:I, j=1:J], gamma[j]>= r[j,i]-mu[j,i]);

    @constraint(model, [i = 1:I, j = 1:J], max_d*(1-z[j,i]) >= mu[j,i]);

    @objective(model, Min, sum(gamma[j] for j=1:J));

    optimize!(model);

    return value.(x), [argmax(value.(z)[i,:]) for i = 1:J], value.(gamma)  # centers, assignments, obj_value
end

function euclidean_optimal_kmenas(points,  I, J, D)
    
    model = JuMP.Model(Gurobi.Optimizer);

    max_d = maximum(euclidean_distance(points));

    set_optimizer_attribute(model, "TimeLimit", 300)

    @variable(model, gamma[1:J]);
    @variable(model, z[1:J, 1:I], Bin);
    @variable(model, mu[1:J, 1:I] >=0);
    @variable(model, r[1:J, 1:I] >=0);
    @variable(model, max_d >= x[1:I, 1:D] >=0);
    @variable(model, y[1:J, 1:D, 1:I] >=0);

    @constraint(model, [j = 1:J], sum(z[j,:]) == 1);

    @constraint(model, [i=1:I, j=1:J, d=1:D], sum(y[j,d,i] for d=1:D) <= r[j,i]);
    @constraint(model,[i=1:I, j=1:J, d=1:D], [y[j,d,i]; x[i,d] - points[j,d]] in SecondOrderCone())

    @constraint(model, [i=1:I, j=1:J], gamma[j]>= r[j,i]-mu[j,i]);

    @constraint(model, [i = 1:I, j = 1:J], max_d*(1-z[j,i]) >= mu[j,i]);

    @objective(model, Min, sum(gamma[j] for j=1:J));

    optimize!(model);

    return value.(x), [argmax(value.(z)[i,:]) for i = 1:J], value.(gamma)  # centers, assignments, obj_value
end

function mean_silhouette_score(assignment, counts, points)
    distances = pairwise(Euclidean(), points')
    return mean(silhouettes(assignment, counts, distances))
end

function euclidean_distance(points)
    n,m = size(points)
    distances = ones((n,n))
    for i in 1:n
        for j in 1:n
            distances[i,j] = sqrt(sum((points[i,:] - points[j,:]).^2))
        end
    end
    return distances
end

function manhattan_distance(points)
    n,m = size(points)
    distances = ones((n,n))
    for i in 1:n
        for j in 1:n
            distances[i,j] = sum(abs.(points[i,:] - points[j,:]))
        end
    end
    return distances
end

function manhattan_warm_start(points, I, J, D)
    
    model = JuMP.Model(Gurobi.Optimizer);

    max_d = maximum(manhattan_distance(points))

    set_optimizer_attribute(model, "TimeLimit", 300)

    @variable(model, gamma[1:J]);
    @variable(model, z[1:J, 1:I],Bin);
    @variable(model, mu[1:J, 1:I] >=0);
    @variable(model, r[1:J, 1:I] >=0);
    @variable(model, max_d >= x[1:I, 1:D] >=0);
    @variable(model, y[1:J, 1:D, 1:I] >=0);

    km = kmeans(points', I);
    x_warm = km.centers';
    z_warm = zeros(J,I);
    assignments = km.assignments;
    for i = 1:J
        z_warm[i,assignments[i]] = 1
    end

    set_start_value.(z, z_warm);
    set_start_value.(x, x_warm);

    @constraint(model, [j = 1:J], sum(z[j,:]) == 1);

    @constraint(model, [i=1:I, j=1:J, d=1:D], sum(y[j,d,i] for d=1:D) <= r[j,i]);
    @constraint(model, [i=1:I, j=1:J, d=1:D], y[j,d,i] >= x[i,d] - points[j,d]);
    @constraint(model, [i=1:I, j=1:J, d=1:D], y[j,d,i] >= -(x[i,d] - points[j,d]));

    @constraint(model, [i=1:I, j=1:J], gamma[j]>= r[j,i]-mu[j,i]);

    @constraint(model, [i = 1:I, j = 1:J], max_d*(1-z[j,i]) >= mu[j,i]);

    @objective(model, Min, sum(gamma[j] for j=1:J));

    optimize!(model);

    return value.(x), [argmax(value.(z)[i,:]) for i = 1:J], value.(gamma)  # centers, assignments, obj_value
end


function euclidean_warm_start(points,  I, J, D)
    
    model = JuMP.Model(Gurobi.Optimizer);

    max_d = maximum(euclidean_distance(points));

    set_optimizer_attribute(model, "TimeLimit", 300)

    @variable(model, gamma[1:J]);
    @variable(model, z[1:J, 1:I], Bin);
    @variable(model, mu[1:J, 1:I] >=0);
    @variable(model, r[1:J, 1:I] >=0);
    @variable(model, max_d >= x[1:I, 1:D] >=0);
    @variable(model, y[1:J, 1:D, 1:I] >=0);

    km = kmeans(points', I);
    x_warm = km.centers';
    z_warm = zeros(J,I);
    assignments = km.assignments;
    for i = 1:J
        z_warm[i,assignments[i]] = 1
    end

    @constraint(model, [j = 1:J], sum(z[j,:]) == 1);

    @constraint(model, [i=1:I, j=1:J, d=1:D], sum(y[j,d,i] for d=1:D) <= r[j,i]);
    @constraint(model,[i=1:I, j=1:J, d=1:D], [y[j,d,i]; x[i,d] - points[j,d]] in SecondOrderCone())

    @constraint(model, [i=1:I, j=1:J], gamma[j]>= r[j,i]-mu[j,i]);

    @constraint(model, [i = 1:I, j = 1:J], max_d*(1-z[j,i]) >= mu[j,i]);

    @objective(model, Min, sum(gamma[j] for j=1:J));

    optimize!(model);

    return value.(x), [argmax(value.(z)[i,:]) for i = 1:J], value.(gamma)  # centers, assignments, obj_value
end


euclidean_warm_start (generic function with 1 method)

## Easy problem (var 1)

In [3]:
"""
J is the number of points/record in the dataset
I is the number of clusters
D is the dimensionality of the points (number of features)
"""

dim2_optimal_performance = []
dim2_kmeans_performance = []

for J in [20,50,100,300,500]
    for I in [2,3,4,5]
        for D = 2
            points = generate_points(I,J,D, ones(I));
            centroids, assignment, obj_value = manhattan_warm_start(points, I,J,D);
            #km = kmeans(points', I);
            push!(dim2_optimal_performance, mean_silhouette_score(assignment, counts(assignment), points))
            #assignments = km.assignments;
            #r = km.counts;
            #push!(dim2_kmeans_performance, mean_silhouette_score(assignments,r, points))
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 340 rows, 224 columns and 800 nonzeros
Model fingerprint: 0x50c6829f
Variable types: 184 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [2e-02, 2e+00]

User MIP start produced solution with objective 3.72261 (0.00s)
Loaded user MIP start with objective 3.72261

Presolve removed 170 rows and 124 columns
Presolve time: 0.00s
Presolved: 170 rows, 100 columns, 402 nonzeros
Found heuristic solution: objective 3.7007202
Variable types: 80 continuous, 20 integer (20 binary)

Root relaxation: objective -3.769919e+00, 112 iterations, 0.00 seconds 

In [5]:
dim2_optimal_performance = rename!(DataFrame(reshape(dim2_optimal_performance, (4,5)), :auto),:x1 => :"20", :x2 => :"50", :x3 => :"100", :x4 => :"300", :x5 => :"500")
dim2_optimal_performance[!,:centers] = ["2","3","4","5"];

#dim2_kmeans_performance = rename!(DataFrame(reshape(dim2_kmeans_performance, (4,5)), :auto),:x1 => :"20", :x2 => :"50", :x3 => :"100", :x4 => :"300", :x5 => :"500")
#dim2_kmeans_performance[!,:centers] = ["2","3","4","5"];


#CSV.write("Results/dim2_kmeans_performance.csv",dim2_kmeans_performance)

LoadError: MethodError: no method matching reshape(::DataFrame, ::Tuple{Int64, Int64})
[0mClosest candidates are:
[0m  reshape([91m::OffsetArrays.OffsetArray[39m, ::Tuple{Vararg{Int64, N}} where N) at /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/OffsetArrays/WvkHl/src/OffsetArrays.jl:380
[0m  reshape([91m::OffsetArrays.OffsetArray[39m, ::Tuple{Vararg{Union{Colon, Int64}}}) at /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/OffsetArrays/WvkHl/src/OffsetArrays.jl:384
[0m  reshape([91m::OffsetArrays.OffsetArray[39m, ::Tuple{Union{Integer, Base.OneTo}, Vararg{Union{Integer, Base.OneTo}}}) at /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/OffsetArrays/WvkHl/src/OffsetArrays.jl:379
[0m  ...

In [8]:
dim2_optimal_performance

Row,20,50,100,300,500,centers
,Any,Any,Any,Any,Any,String
1,0.756077,0.812402,0.822535,0.827132,0.831849,2
2,0.749356,0.740422,0.770554,0.777486,0.76352,3
3,0.699222,0.627198,0.664542,0.698736,0.633263,4
4,0.593767,0.571434,0.616349,0.646467,0.649447,5


In [7]:
CSV.write("Results/dim2_warm_start_performance.csv",dim2_optimal_performance)

"Results/dim2_warm_start_performance.csv"